# <center>Linear Regression</center>

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Anaconda native modules
from IPython.display import display, Markdown, Latex
from tabulate import tabulate
from statistics import NormalDist, variance

# Data extraction and partition

In [ ]:
# Input data
# Source: MSc. Nguyen Ngoc, Toan
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Extracting data features and targeting dependent variable ground truth values for training and testing
X_train = train.iloc[:, :-1]    # Dataframe (Containing 5 training features)
y_train = train.iloc[:, -1]     # Series    (Containing 1 training target)

X_test = test.iloc[:, :-1]      # Dataframe (Containing 5 testing features)
y_test = test.iloc[:, -1]       # Series    (Containing 1 testing target

# A helper list to stringify feature descriptions
feature_descriptions = ['Hours Studied', 'Previous Scores', 'Extracurricular Activities Participation', 'Sleep Hours', 'Sample Question Papers Practiced,']

# Common procedures

In [ ]:
# Lab 4's template OLS Linear Regression model
class OLSLinearRegression:
    w = np.ndarray
    ridge_lambda: float | None = None

    def fit(self, X, y):
        ''' 
        This function is used to fit the model to the data. It uses the Ordinary Least Squares method to find the optimal parameters.

        Parameters
        ----------
        X : np.array
            Input data
        y : np.array
            Output data

        Returns
        -------
        self : object
            Returns the instance of the class
        
        '''
        X = np.c_[ X, np.ones(X.shape[0]) ] # appending a constant of unexplained parameter
        if self.ridge_lambda is not None:
            X_pinv = np.linalg.inv(X.T @ X + self.ridge_lambda*np.eye(X.shape[1])) @ X.T # see reference
        else:
            X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        ''' 
        This function is used to get the parameters of the model.

        Returns
        -------
        self.w : np.array
            Optimal parameters (column vector)
        '''

        return self.w


    def predict(self, X):
        ''' 
        This function is used to predict the output of the model.

        Parameters
        ----------
        X : np.array
            Input data

        Returns
        -------
        X @ self.w : np.array
            Predicted output
        '''
        X = np.c_[ X, np.ones(X.shape[0]) ] # appending a constant of unexplained parameter
        return X @ self.w
    
    def latex_repr(self, centered=False) -> Latex:
        model_repr = ('$$' if centered else '$') + '\\hat{{y}}='
        params = self.get_params()
        for i in range(len(params)):
            model_repr += '-' if params[i] < 0 else ('+' if i > 0 else '')
            if i > 0:
                model_repr += ' '
            model_repr += '{param:.3f}'.format(param=np.abs(params[i]))
            if(i < len(params) - 1):
                model_repr += 'x_{id} '.format(id=i+1)
        model_repr += '$$' if centered else '$'
        return Latex(model_repr)
    

def ols_fitness_stats(M: OLSLinearRegression, X, Y, print_statistic=False) -> tuple[float, float, float, float, float]:
    """
    Validating the model using a testing set and returns 
            Mean (Summed) Absolute error, 
            Max Error, 
            Min Error, 
            Mean of Error, 
            Error Variance
    statistics.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The ordered testing data points. N is the number of data points, n is the number of features.
    Y : Array-like, np.ndarray, shape (N)
        The ground truth data of each ordered testing data points.
    print_statistic : boolean
        Print the returning statistic.

    Returns
    ----------
    MAE : float
        The model's Mean (Summed) Absolute Error.
    MxE : float
        The model's MaXimum absolute Error.
    MnE : float
        The model's MiNimum absolute Error.
    EM : float
        The model's Error Mean.
    Var : float
        The model's error Variance.
    """
    Err = Y.ravel()-M.predict(X).ravel()
    AE = np.abs(Err)
    EM = np.mean(Err)
    MxE = np.max(AE)
    MnE = np.min(AE)
    test_len = Y.shape[0]
    Var = variance(Err)

    MAE = np.sum(AE)/test_len
    if print_statistic:
        display(Latex('$\\text{{MAE}}: {mae:.3f}$'.format(mae=MAE)))
        display(Latex('$\\text{{max absolute error: }}{maxerr:.3f}$'.format(maxerr=MxE)))
        display(Latex('$\\text{{min absolute error: }}{minerr:.3f}$'.format(minerr=MnE)))
        display(Latex('$\\text{{Error mean: }}{varr:.3f}$'.format(varr=EM)))
        display(Latex('$\\text{{Error variance: }}{varr:.3f}$'.format(varr=Var)))
    return MAE, MxE, MnE, EM, Var


# Data exploration

In [ ]:
# Data exploration on training set

# preprocessing
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

if(X_train.ndim < 2):
    X_train = X_train.reshape(X_train.shape + (1,))
if(X_test.ndim < 2):
    X_test = X_test.reshape(X_test.shape + (1,))

def linear_regression_correlation(a, b):
    """
    Single variable regression analysis

    Parameters
    ----------
    a, b : Array-like, np.ndarray shape (N)
        The input independent and dependent variable vectors respectively.
        The assumed regression model is the simple linear regression model: y = b1 + b0.
    
    Returns
    ----------
    b1 : float
        The coefficient associated with the independent variable.
    b0 : float
        The constant terms of unexplained value of the model
    r : float range [-1, 1]
        The correlation coefficient, representing the explainability of b when depending on a.
    """
    mean_x = a.sum()/a.shape[0]
    mean_y = b.sum()/b.shape[0]
    Sxy = np.sum((a-mean_x)@(b-mean_y))
    Sxx = np.sum(np.square(a-mean_x))
    Syy = np.sum(np.square(b-mean_y))
    return (Sxy/Sxx, mean_y-(Sxy/Sxx)*mean_x, Sxy/np.sqrt(Sxx*Syy))

def multivariable_approximate_curvature(X, Y, singular_color=None, singular_index=None):
    """
    Plotting the independence variable(s) array x in Re^(d=n) and the resulting y in Re,
    along with a linear regression line of the data pairs.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        Array of input data points independence variable(s).
    Y : Array-like, np.ndarray, shape (N)
        Array of scalar response variable corresponds to the input data points.
    singular_color : str
        The single hex value representing the RGB color of the graph's marker.

    Returns
    ----------
    None
    """
    fig, ax = plt.subplots()
    
    for i in range(X.shape[1]):
        x_p = X[:, i]
        linregr = linear_regression_correlation(x_p, Y)
        y_tilde = linregr[1] + linregr[0]*x_p
        if singular_color is None:
            color = ax._get_lines.get_next_color()
        else:
            color = singular_color
        ax.plot(x_p, Y, 'o', color=color, label= ('x{idx}'.format(idx=i+1) if singular_index is None else f'x{singular_index}') + ', r: {rv:.7f}'.format(rv=linregr[2]) )
        ax.plot(x_p, y_tilde, '-', color=color)
        ax.set_xlabel('x' if singular_index is None else f'x{singular_index}')
        ax.set_ylabel('y')
        ax.set_title('Multivariable approximate curvature')
    ax.legend()

""" Plotting set
for i in range(X_train.shape[1]+1):
    if i == X_train.shape[1]:
        multivariable_approximate_curvature(X_train[:, :], y_train[:])
        continue
    else:
        color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
        multivariable_approximate_curvature(X_train[:, i, np.newaxis], y_train[:], color_list[i], i+1)
# """

def histogram_analysis(x):
    """
    Plotting the histogram of a variable's distribution

    Parameters
    ----------
    x : Array-like, np.ndarray, shape (N)
        Array of input data points univariable feature, i.e xi in Re.
    
    Returns
    ----------
    None
    """
    fig, ax = plt.subplots()
    ax.hist(x)
    ax.set_title('Variable distribution histogram')
    ax.legend()
    plt.draw()
    return fig, ax

""" Plotting set
for i in range(X_train.shape[1]]):
    fig, ax = histogram_analysis(X_train[:, i])
    fig.suptitle('x{i}'.format(i=i+1))
# """

def bivariate_analysis(X, p):
    """
    Plotting the the correlation between all none-ordered pairs of (p, i) for i <> p and i in [1, n], n is the number of features

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        Array of input data points independence variable(s).
    p : integer, range [1, n]
    
    Returns
    ----------
    None
    """
    for i in range(X.shape[1]):
        if((i+1) == p):
            continue
        fig, ax = plt.subplots()
        ax.scatter(X[:, p-1], X[:, i])
        ax.set_xlabel(f'x{p}')
        ax.set_ylabel(f'x{i+1}')
        ax.set_title(f'Bivariate scatter plot of x{p} and x{i+1}')
    plt.draw()

""" Plotting set
for i in range(X_train.shape[1]):
    bivariate_analysis(X_train[:], i+1)
# """;

# Multivariate, simple, rank 1 polynomial linear regression of all features

In [ ]:
def ols_linregress(X, Y, print_model=True) -> OLSLinearRegression:
    """
    Perform an Ordinary Least Squared Linear Regression using the independence variables as the basis linear function, having a maximum of 1 input variate nad
    with a maximum polynomial level of 1 and f_i: R->R. Hence, this also is a linear function.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The ordered training data points. N is the number of data points, n is the number of features.
    Y : Array-like, np.ndarray, shape (N)
        The ground truth data of each ordered data points.
    print_model : boolean
        Print the model mathematical expression.
        
    Returns
    ----------
    model : OLSLinearRegression
        An OLSLinearRegression object wrapping the model's architecture and trained parameters.
    """
    assert X.shape[0] == Y.shape[0] # making sure they inputs have the same data point sizes
    # we mostly takes input as vectors of features, so to model it, we must also consider the constant term of the model.
    # Hence, we stack another column after the last column of X
    model = OLSLinearRegression()
    model.fit(X, Y)
    if print_model:
        display(model.latex_repr(True))
    return model

In [ ]:
# Calculating the model's MAE
mdl = ols_linregress(X_train, y_train)
ols_fitness_stats(mdl, X_test, y_test, True)

# """ Plot the resulting model
fig = plt.figure()
ax = fig.add_subplot()
mark_truth = ax.plot(X_test, y_test, 'ro')
mark_predict = ax.plot(X_test, mdl.predict(X_test), 'bo')
ax.set_xlabel(f'x')
ax.set_ylabel(f'y')
ax.legend(handles=[mark_truth[0], mark_predict[0]], labels=['Ground truth data', 'Model prediction'])
ax.set_title('Linear regression by OLS model based on all feature', loc="center")
plt.draw()
# """

$$
\text{Student Performance} = 2.852x_1 + 1.018x_2 + 0.604x_3 + 0.474x_4 + 0.192x_5 - 33.969
$$
Where $x_1$ is the hours studied, $x_2$ is the previous scores, $x_3$ is the extracurricular activities participation, $x_4$ is the sleep hours and $x_5$ is the number of sample question papers practied.

# Single variable, simple, rank 1 polynomial linear regression for each feature

Models are trained under <i>k</i>-fold Cross validation method.

In [ ]:
# Performing k-fold training for each model, then compared them using MAE and selecting the best model for further operations.

def k_fold(X, Y, k, shuffle=False) -> tuple[OLSLinearRegression, float]:
    """
    Perform K-fold cross validation process on covariate matrix X and response matrix Y, 
    returning a parameter-wise averaging model and the averaged MAE fitness value

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The covariate training matrix.
    Y : Array-like, np.ndarray, shape (N)
        The ground truth training response vector.
    k : integer
        The number of partition (fold) to be considered.
    shuffle: boolean
        Specify whether or not the procedure should perform a one-time permutation of the training set.

    Returns
    ----------
    avg_model : OLSLinearRegression
        A parameter-wise averaged OLSLinearRegression model after training for k cross validation processes.
    MAE : float
        The averaged MAE fitness meter after k cross validation processes.
    """
    k=int(k)
    # not really used very often, just an utility
    if shuffle:
        assert X.shape[0] == Y.shape[0] # making sure they inputs have the same data point sizes
        shuffle_indices = np.random.default_rng().permutation(X.shape[0])
        X = X[shuffle_indices]
        Y = Y[shuffle_indices]

    
    data_steps = int(X.shape[0] / k)
    cross_validate_fitness = 0
    model_params = np.zeros(X.shape[1]+1)
    for k_step in range(k):
        # overshadowing X_train defined in global space by the local k-th step training set
        # No need to append a constant N size 1-vector here since ols_linregress already did that for us
        X_train = np.r_[ X[0:data_steps*k_step], X[data_steps*(k_step+1):] ]
        Y_train = np.r_[ Y[0:data_steps*k_step], Y[data_steps*(k_step+1):] ]
        subsample_linear_model = ols_linregress(X_train, Y_train, False)

        # Validation set
        X_validate = X[data_steps*k_step:data_steps*(k_step+1)]
        Y_validate = Y[data_steps*k_step:data_steps*(k_step+1)]

        k_MAE = ols_fitness_stats(subsample_linear_model, X_validate, Y_validate)[0]
        cross_validate_fitness += k_MAE
        model_params += subsample_linear_model.get_params()

    # Averaging after k-th steps of training and assign the averaged results to the model
    linear_model = OLSLinearRegression()
    model_params = model_params/k
    linear_model.w = model_params
    return linear_model, cross_validate_fitness / k

def single_feature_k_fold_ols(X, Y, k) -> tuple[list[OLSLinearRegression], np.ndarray]:
    """
    Perform an Ordinary Least Squared Linear Regression on n model f_i: R->R, f_i(x_i) = a_i*x_i + b_i Where i is the i-th feature of the independence variable set;
    and returns a list of OLSLinearRegression model and an array of MAE of each model. Both have length corresponding to n features.


    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The ordered training variables. N is the number of data points, n is the number of features.
    Y : Array-like, np.ndarray, shape (N)
        The ground truth data of each ordered data points.
    k : integer
        The number of training data fold (partition).
    
    Returns
    ----------
    models : list[OLSLinearRegression]
        A list of OLSLinearRegression object wrapping the models' architectures and trained parameters.
    MAE_list : np.ndarray((k,))
        An 0-indexed array of averaged out MAE fitness corresponds to each i-th model in models.
    """
    assert X.shape[0] == Y.shape[0] # making sure they inputs have the same data point sizes
    shuffle_indices = np.random.default_rng().permutation(X.shape[0])
    X_shuffle = X[shuffle_indices]
    Y_shuffle = Y[shuffle_indices]
    models = []
    fitnesses = np.ndarray(X.shape[1])

    for i in range(X.shape[1]):
        X_feature = X_shuffle[:, i, np.newaxis]
        Y_feature = Y_shuffle # since our Y is scalar-typed vector
        subsample_model, subsample_fitness = k_fold(X_feature, Y_feature, k)
        # saving results
        models.append(subsample_model)
        # MAE Mean
        fitnesses[i] = subsample_fitness
    return models, fitnesses

# cross validate results
ms, fitnesses = single_feature_k_fold_ols(X_train, y_train, 10)
# cross validation data representation
models_repr = []
for i, (m, fitness) in enumerate(zip(ms, fitnesses)):
    models_repr.append('$\hat{{y}}={b1:.3f}x_{i}'.format(i=i+1, b1=m.get_params()[0]) \
                  + ('+ {b0:.3f}$' if m.get_params()[1] > 0 else '{b0:.3f}$').format(b0=m.get_params()[1]))
cross_validation_output = {
    'Index': range(1,len(ms)+1),
    'Feature Description': feature_descriptions,
    'Model' : models_repr,
    'MAE': np.round(fitnesses, 3)
}
cv_table = tabulate(cross_validation_output, headers='keys', tablefmt='github')
display(Markdown('<center>\n\n' + cv_table + '\n\n</center>'))

# Extracting the index of the best feature (since we have been following the structural run of the codebase, 
# one can assumes that the index of this list aligns with the index of the feature)
best_index = np.argmin(fitnesses)
display(Markdown('<center>\n\nBest candidate feature: ' + feature_descriptions[best_index] + '\n\n</center>'))

In [ ]:
# Training the best model

X_best_train = X_train[:, best_index]
# Best fit polynomial of degree 1 OLS Linear Regression model
best_feature_model = ols_linregress(X_best_train, y_train, False)
display(Latex('$$\hat{{y}}={b1:.3f}x_{i}'.format(i=best_index+1, b1=best_feature_model.get_params()[0]) \
                  + ('+ {b0:.3f}$$' if best_feature_model.get_params()[1] > 0 else '{b0:.3f}$$').format(b0=best_feature_model.get_params()[1])))
# """ Plot the resulting model
fig = plt.figure()
ax = fig.add_subplot()

ax.plot(X_test[:, best_index], y_test, 'ro', label='Ground truth data')
ax.plot(X_test[:, best_index], best_feature_model.predict(X_test[:, best_index]), 'b-', label='Model prediction')
ax.set_xlabel(f'x{best_index+1}')
ax.set_ylabel(f'y')
ax.legend()
ax.set_title('Linear regression by OLS model based on ' + feature_descriptions[best_index] + ' feature', loc="center")
plt.draw()
# """


In [ ]:
display(Latex('$$\\text{{MAE:}} \quad {mae:.3f}$$'.format(mae=ols_fitness_stats(best_feature_model, X_test[:, best_index], y_test)[0])))

$$\text{Student Performance} = 1.011x_2+1.011$$

# Complex linear models

## Model definitions

In [ ]:
# hypothesis testing

def hypothesis_test_feature_contribution(X, Y, X_test, Y_test, alpha) -> tuple[float, bool]:
    """
    Perform hypothesis tesing by p-value of the simple linear regression model's coefficient b1 closing in on 0 
    of a linear model with input of assumed independent univariable X, and scalar response vector Y

    Note: a true evident doesn't really says much about the true facts of the feature, since outliers and missing data
    could interfere with the process. But for the sakes of simplicity, it is an acceptable result that if
    evident is True, then we can conclude the feature has significant contribution to the model, given a significance level alpha

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, 1)
        The input training variable
    Y : Array-like, np.ndarray, shape (n)
        The input training response (ground truth) vector
    X_test : Array-like, np.ndarray, shape (N, 1)
        The input testing variable
    Y_test : Array-like, np.ndarray, shape (n)
        The input testing response (ground truth) vector
    alpha : float
        The significance level of the test

    Returns
    ----------
    p : float
        The statistic p-value, assuming a 
    evident : boolean
        Whether the test yields enough evidence to prove the significance of the feature
    """
    lin_regress_model_test = OLSLinearRegression()
    lin_regress_model_test.fit(X, Y)
    MSE = np.sum(np.square(Y_test.ravel()-lin_regress_model_test.predict(X_test).ravel()))/(X_test.shape[0])
    
    # Assuming the b1 converges to a normal distribution given our large dataset
    X_test_mean = np.mean(X_test)
    #sigma_b0 = np.sqrt(MSE)*np.sqrt((1/X_test.shape[0])+np.square(X_test_mean)/Sxx)  no need, since we only need b1 for linearlity test
    sigma_b1 = np.sqrt(MSE/np.sum(np.square(X_test - X_test_mean)))
    z0 = lin_regress_model_test.get_params()[0]/(sigma_b1)
    p_value = 2*(1-NormalDist().cdf(np.abs(z0)))
    return [p_value, p_value <= alpha]

# performing hypothesis test on each feature
def perform_hypothesis_test():
    """ 
    Perform simple hypothesis testing on each feature's univariate simple linear regression model slope coefficient
    alpha = 0.5

    Parameters
    ----------
    None

    Returns
    ----------
    None
    """
    alpha = 0.05
    for i in range(X_train.shape[1]):
        p_value, evident = hypothesis_test_feature_contribution(X_train[:, i], y_train, X_test[:, i], y_test, alpha=alpha)
        print(f"alpha: {alpha}, p-value: {p_value}, evident: {evident}")

# y = ax1 + bx2
def model_1_expression(X):
    """
    C Model 1 matrix representation.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.

    Returns
    ----------
    X' : Array-like, np.ndarray, shape (N, 2)
        The resulting model's covariate matrix.
    """
    return X[:, :2]

def model_1(X, Y, **kwargs):
    """
    Construct a C Model 1 OLS Linear Regression object.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.
    Y : Array-like, np.ndarray, Shape (N)
        The response vector.

    Returns
    ----------
    model : OLSLinearRegression
        C Model 1 object wrapper.
    """
    X = model_1_expression(X)
    model = OLSLinearRegression()
    model.fit(X, Y)
    return model

# ridge regression by L2 regularization form
# multivariable_approximate_curvature(X_train[:, 0, np.newaxis]*X_train[:, 1, np.newaxis], y_train[:])
# https://en.wikipedia.org/wiki/Anscombe_transform#Alternatives
# https://en.wikipedia.org/wiki/Ridge_regression#Generalized_Tikhonov_regularization
# y = 2asqrt(x1*x2+3/8) + b*x2 + c*x3 + d*(x4/x1) + e*x5 + err
def model_2_expression(X):
    """
    C Model 2 matrix representation.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.

    Returns
    ----------
    X' : Array-like, np.ndarray, shape (N, 5)
        The resulting model's covariate matrix.
    """
    X1X2 = X[:, 0]*X[:, 1]
    return np.c_[ 2*np.sqrt(X1X2 + 3/8), X[:, 1:3], X[:, 3]/X[:, 0], X[:, 4]]

def model_2(X, Y, ridge_lambda, **kwargs):
    """
    Construct a C Model 2 OLS Linear Regression object while applying L2 Regularization with Tikhonov factor = ridge_lambda.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.
    Y : Array-like, np.ndarray, Shape (N)
        The response vector.
    ridge_lambda : float
        The Tikhonov factor used to bias the model coefficient in case of nonorthogonal covariate feature matrix

    Returns
    ----------
    model : OLSLinearRegression
        C Model 2 object wrapper.
    """
    X = model_2_expression(X)
    model = OLSLinearRegression()
    model.ridge_lambda = ridge_lambda
    model.fit(X, Y)
    return model

# y = a*x1 + b*x2 + c*( ( Norm(x1)/std_dev(Norm(x1)) ) * Norm(x2) ) + d*x3 + e*(1-x1/x4) + f*x5 + err
def model_3_expression(X):
    """
    C Model 3 matrix representation.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.

    Returns
    ----------
    X' : Array-like, np.ndarray, shape (N, 6)
        The resulting model's covariate matrix.
    """
    X1_norm = (X[:, 0] - np.mean(X[:, 0]))/np.sqrt(variance(X[:, 0]))
    X2_norm = (X[:, 1] - np.mean(X[:, 1]))/np.sqrt(variance(X[:, 1]))
    X1_std_dev = np.sqrt(np.sum(np.square(X1_norm-np.mean(X1_norm)))/X1_norm.shape[0])
    return np.c_[ X[:, 0], X[:, 1], (X1_norm/X1_std_dev)*X2_norm, X[:, 2], (1-X[:, 0]/X[:, 3]), X[:, 4] ]

def model_3(X, Y, **kwargs):
    """
    Construct a C Model 3 OLS Linear Regression object.

    Parameters
    ----------
    X : Array-like, np.ndarray, shape (N, n)
        The full-size feature matrix.
    Y : Array-like, np.ndarray, Shape (N)
        The response vector.

    Returns
    ----------
    model : OLSLinearRegression
        C Model 3 object wrapper.
    """
    X = model_3_expression(X_train)
    model = OLSLinearRegression()
    model.fit(X, Y)
    return model

## Models testing

In [ ]:
# Performing linear regression for each custom model, printing out the resulting MAE and select the best model for further operations.

shuffle_indices = np.random.default_rng().permutation(X_train.shape[0])
X_shuffle = X_train[shuffle_indices]
Y_shuffle = y_train[shuffle_indices]
c_model = [model_1_expression, model_2_expression, model_3_expression]
c_model_k_fold = [k_fold(c_model[0](X_shuffle), Y_shuffle, 10), 
                  k_fold(c_model[1](X_shuffle), Y_shuffle, 10),
                  k_fold(c_model[2](X_shuffle), Y_shuffle, 10)
                  ]
c_gen_model = [model_1, model_2, model_3]
c_models_repr = [
    ('$$\hat{{y}}={:.3f}x_1{}{:.3f}$$'),
    ('$$\hat{{y}}=2\\times {:.3f}\sqrt{{x_1x_2+\dfrac{{3}}{{8}}}}' + \
         '{}{:.3f}x_2' + \
         '{}{:.3f}x_3' + \
         '{}{:.3f}\dfrac{{x_4}}{{x_1}}' + \
         '{}{:.3f}x_5' + \
         '{}{:.3f}$$ '),
    ('$$\hat{{y}}={:.3f}x_1' + \
         '{}{:.3f}x_2' + \
         '{}{:.3f}\dfrac{{{{Norm}}(x_1)}}{{\sigma_{{{{Norm}}(x_1)}}}}{{Norm}}(x_2)' + \
         '{}{:.3f}x_3' + \
         '{}{:.3f}\dfrac{{x_1}}{{x_4}}' + \
         '{}{:.3f}x_5' + \
         '{}{:.3f}$$ ')
]

# formating model 2 and model 3 representation
format_1 = ['']*(c_model_k_fold[1][0].get_params().shape[0]*2-1)
format_1[0::2] = list(c_model_k_fold[1][0].get_params())
format_1[1:(c_model_k_fold[1][0].get_params().shape[0]*2-2):2] = \
    ['+' if c_model_k_fold[1][0].get_params()[i] >= 0 else '' for i in range(1, c_model_k_fold[1][0].get_params().shape[0])]

format_2 = ['']*(c_model_k_fold[2][0].get_params().shape[0]*2-1)
format_2[0::2] = list(c_model_k_fold[2][0].get_params())
format_2[1:(c_model_k_fold[2][0].get_params().shape[0]*2-2):2] = \
    ['+' if c_model_k_fold[2][0].get_params()[i] >= 0 else '' for i in range(1, c_model_k_fold[2][0].get_params().shape[0])]

# tabulating data
cross_validation_output = {
    'Index': range(1,4),
    'Model': [
        c_models_repr[0].format(c_model_k_fold[0][0].get_params()[0],
                                '+' if c_model_k_fold[0][0].get_params()[1] >= 0 else '',
                                 c_model_k_fold[1][0].get_params()[1]),
        c_models_repr[1].format(*format_1) +
            'Parameters regression and regularization: $$\hat{{\\beta}}=({{\mathbf{{X}}}}^{{\\top}}\mathbf{{X}}+\
            \lambda \mathbf{{\mathit{{I}}}}){{\mathbf{{X}}}}^{{\\top}}\mathbf{{y}}$$',
        c_models_repr[2].format(*format_2) +
         '$$\\text{{Where}}\quad {{Norm}}(U)=\\dfrac{{U-\overline{{U}}}}{{\sigma_U}}, \quad \
            \sigma_U= \\sqrt{{\\dfrac{{\sum_{{u\in U}} (u-\\overline{{U}})^2}}{{\|U\|}}}}$$'
    ],
    'MAE': np.round(np.array([c_model_k_fold[0][1], c_model_k_fold[1][1], c_model_k_fold[2][1]]), 3)
}
cv_table = tabulate(cross_validation_output, headers='keys', tablefmt='github')
display(Markdown('<center>\n\n' + cv_table + '\n\n</center>'))

# Extracting the index of the best feature (since we have been following the structural run of the codebase, 
# one can assumes that the index of this list aligns with the index of the feature)
best_index = np.argmin([c_model_k_fold[0][1], c_model_k_fold[1][1], c_model_k_fold[2][1]])
format_best = ['']*(c_model_k_fold[best_index][0].get_params().shape[0]*2-1)
format_best[0::2] = list(c_model_k_fold[best_index][0].get_params())
format_best[1:(c_model_k_fold[best_index][0].get_params().shape[0]*2-2):2] = \
    ['+' if c_model_k_fold[2][0].get_params()[i] >= 0 else '' for i in range(1, c_model_k_fold[2][0].get_params().shape[0])]

display(Markdown(f'<center>\n\nBest candidate model: {c_models_repr[best_index].format(*format_best)}\n\n</center>'))
# remember to normalize test input!

In [ ]:
# Training the best model

X_best_train = X_train
# Best fit polynomial of degree 1 OLS Linear Regression model
# The keyword ridge_lambda presents here to accomodate model's ridge_lambda parameter,
# since all model procedure already included the **kwargs parameter, they can safely ignore it if not needed
my_best_model = c_gen_model[best_index](c_model[best_index](X_train), y_train, ridge_lambda=0.01)
format_best = ['']*(my_best_model.get_params().shape[0]*2-1)
format_best[0::2] = list(my_best_model.get_params())
format_best[1:(my_best_model.get_params().shape[0]*2-2):2] = \
    ['+' if my_best_model.get_params()[i] >= 0 else '' for i in range(1, my_best_model.get_params().shape[0])]
display(Markdown(f'<center>\n\nBest model: {c_models_repr[best_index].format(*format_best)}\n\n</center>'))

# """ Plot the resulting model
fig = plt.figure()
ax = fig.add_subplot()

handle_1 = ax.plot(X_test, y_test, 'ro', label='Ground truth data')
handle_2 = ax.plot(X_test, my_best_model.predict(c_model[best_index](X_test)), 'bo', label='Model prediction')
ax.set_xlabel(f'x')
ax.set_ylabel(f'y')
ax.legend(handles=[handle_1[0], handle_2[0]])
ax.set_title('Linear regression by OLS model ' + str(best_index+1), loc="center")
plt.draw()
# """


In [ ]:
display(Latex('$$\\text{{MAE:}} \quad {mae:.3f}$$'.format(mae=ols_fitness_stats(my_best_model, c_model[best_index](X_test), y_test)[0])))

$$\text{Student Performance} = 3.291x_1+1.018x_22−0.017\dfrac{{Norm}(x_1)}{\sigma_{{Norm}(x_1)}}{Norm}(x_2)+0.595x_3+2.676\dfrac{x1}{x4}+0.194x_5−33.563$$